In [1]:
import numpy as np, pandas as pd, warnings, os, re
import lhapdf, yadism

In [2]:
beam_energy = 10.5473 # In GeV. Taken from RCDB for RGE
proton_mass = 0.938 # In GeV

### Predicting the carbon cross section

#### Loading the RGE data to get the x and Q2 bin centers we want to predict


In [3]:
RGE_file_name = "/home/ryan/clas_analysis/clas12-rge-analysis/analysis/unfolded_deuterium_absolute_cross_sections_pass09"
RGE_df = pd.read_csv(RGE_file_name+".csv")
# Renaming x_bin_center and Q2_bin_center to x and Q2, respectively
RGE_df = RGE_df.rename(columns={
    "x_bin_center":"x",
    "Q2_bin_center":"Q2",
})
# The data is based off of bin centers, so y calculated from these can be unphysical. Remove these points
RGE_df["y"] = RGE_df["Q2"]/(2*proton_mass*beam_energy*RGE_df["x"])
RGE_df = RGE_df.query("y > 0 and y < 1")
target="LD2"

#### Setting up the theory and observable cards


In [4]:
theory_card = {
    "PTO": 1, "FNS": "ZM-VFNS", "DAMP": 0, "ModEv": "TRN", "ModSV": "unvaried",
    "XIR": 1.0, "XIF": 1.0, "IC": 1, "IB": 0, "NfFF": 5, "MaxNfAs": 5, "MaxNfPdf": 5,
    "Q0": 1.65, "alphas": 0.118, "Qref": 91.2, "nf0": 4, "nfref": 5, "QED": 0,
    "alphaqed": 0.007496252, "Qedref": 1.777, "SxRes": 0, "SxOrd": "LL",
    "HQ": "POLE",
    "mc": 1.51, "Qmc": 1.51, "kcThr": 1.0,
    "mb": 4.92, "Qmb": 4.92, "kbThr": 1.0,
    "mt": 172.5, "Qmt": 172.5, "ktThr": 1.0,
    "CKM": "0.97428 0.22530 0.003470 0.22520 0.97345 0.041000 0.00862 0.04030 0.999152",
    "MZ": 91.1876, "MW": 80.398, "GF": 1.1663787e-05, "SIN2TW": 0.23126,
    "TMC": 1, "MP": 0.938, "global_nx": 0, "EScaleVar": 1,
    "kDIScThr": 1.0, "kDISbThr": 1.0, "kDIStThr": 1.0, "n3lo_cf_variation": 0,
}


# Need an interpolation grid for x. Not sure what this is for
try:
    xgrid = __import__("eko").interpolation.lambertgrid(60).tolist()
except Exception:
    xs = RGE_df["x"].to_numpy()
    xmin = max(5e-6, 0.3*np.nanmin(xs))
    xmax = min(0.999, 3.0*np.nanmax(xs))
    xgrid = np.unique(np.r_[np.logspace(np.log10(xmin), np.log10(xmax), 240), xs]).tolist()

observable_card = {
    "prDIS": "NC",
    "ProjectileDIS": "electron",   # e− beam (CLAS era)
    "TargetDIS": "proton",         # per-nucleon PDFs; deuteron handled by PDF set
    "PolarizationDIS": 0.0,
    "PropagatorCorrection": 0.0,
    "NCPositivityCharge": None,
    "interpolation_is_log": True,
    "interpolation_polynomial_degree": 4,
    "interpolation_xgrid": xgrid,
    "observables": {
        "F2_total":[{"x":float(xi),"Q2":float(Qi),"y":float(yi)} for xi,Qi,yi in RGE_df[["x","Q2","y"]].to_numpy()],
        "FL_total":[{"x":float(xi),"Q2":float(Qi),"y":float(yi)} for xi,Qi,yi in RGE_df[["x","Q2","y"]].to_numpy()],
        "F3_total":[{"x":float(xi),"Q2":float(Qi),"y":float(yi)} for xi,Qi,yi in RGE_df[["x","Q2","y"]].to_numpy()],
    },
}


#### Running yadism

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    out = yadism.run_yadism(theory_card, observable_card)

┌────────────────────────────────────┐                                       
                                      │                                    │                                       
                                      │ __     __       _ _                │                                       
                                      │ \ \   / /      | (_)               │                                       
                                      │  \ \_/ /_ _  __| |_ ___ _ __ ___   │                                       
                                      │   \   / _` |/ _` | / __| '_ ` _ \  │                                       
                                      │    | | (_| | (_| | \__ \ | | | | | │                                       
                                      │    |_|\__,_|\__,_|_|___/_| |_| |_| │                                       
                                      │                                    │                                       
                                      └────────────────────────────────────┘

Plan

 • F2_total at 715 pts                                                                                             
 • FL_total at 715 pts                                                                                             
 • F3_total at 715 pts

Calculation

yadism took off! please stay tuned ...

...

[14:24:50] INFO     computing P_qq_0 - took: 0.329719 s                                      ]8;id=957961;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=110206;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py#85\85]8;;\
Starting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

           INFO     computing P_qg_0 - took: 0.263521 s                                      ]8;id=42050;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=913393;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py#85\85]8;;\
Starting... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

[14:24:55] INFO     computing P_qq_0 - took: 0.336651 s                                      ]8;id=515933;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=598603;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py#85\85]8;;\
Computing F2_total ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   8% 0:00:50

[14:24:56] INFO     computing P_qg_0 - took: 0.266519 s                                      ]8;id=368863;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py\scale_variations.py]8;;\:]8;id=405571;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/yadism/esf/scale_variations.py#85\85]8;;\
Computing F2_total ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   8% 0:00:50

took 33.72 s

#### Computing the absolute cross sections from yadism

In [6]:
# Function to get F2, FL, and xF3 from yadism
def extract_Fs(pdf):
    vals = out.apply_pdf(pdf)
    F2  = np.array([v["result"] for v in vals["F2_total"]])
    FL  = np.array([v["result"] for v in vals["FL_total"]])
    xF3 = np.array([v["result"] for v in vals["F3_total"]])  # Yadism returns xF3
    return F2, FL, xF3

# Computing the differential cross section
# See Mark Thomson formula 8.23. Here, FL = F2-2xF1
def compute_cross_section(F2, FL, xF3, x, y, Q2, alpha_em, e_minus=True, wrt="dx"):
    """Return d^2σ/(dQ^2 d{wrt}), wrt ∈ {"dx","dy"} (default: dx)."""
    Yp   = 1 + (1 - y)**2
    Ym   = 1 - (1 - y)**2
    sign = +1 if e_minus else -1           # e− uses +, e+ uses −
    denom = y if (wrt == "dy") else x      # <-- KEY CHANGE
    kin  = (2*np.pi*alpha_em**2) / (np.maximum(denom,1e-12) * np.maximum(Q2,1e-12)**2)  # GeV^-2
    GEV2_TO_PB = 3.89379338e8
    return kin * (Yp*F2 - y*y*FL + sign*Ym*xF3) * GEV2_TO_PB   # pb/GeV^2

if target=="C":
    pdf = "nCTEQ15HIX_FullNuc_12_6"
elif target=="LD2":
    pdf = "nCTEQ15HIX_FullNuc_2_1"
alpha = theory_card.get("alphaqed", 1/137.036)
    
# Getting the cross section at the bin centers using the pdf
central_F2, central_FL, central_xF3 = extract_Fs(lhapdf.mkPDF(pdf))
central_cross_section = compute_cross_section(
    central_F2,
    central_FL, 
    central_xF3,
    RGE_df["x"].to_numpy(),
    RGE_df["y"].to_numpy(),
    RGE_df["Q2"].to_numpy(),
    alpha,
    e_minus=True,
    wrt="dx"
)

LHAPDF 6.5.5 loading /home/ryan/yadism/LHAPDF-6.5.5/install/share/LHAPDF/nCTEQ15HIX_FullNuc_2_1/nCTEQ15HIX_FullNuc_2_1_0000.dat
nCTEQ15HIX_FullNuc_2_1 PDF set, member #0, version 3; LHAPDF ID = 110050


[14:25:23] INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=30828;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=570491;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

[14:25:24] INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=755616;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=780959;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=486596;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=892087;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

#### Computing the 68th percentile for uncertainties

In [7]:
# robust Hessian loop: use only members present
def present_ids(setname):
    ids=set()
    for p in lhapdf.paths():
        d=os.path.join(p,setname)
        if os.path.isdir(d):
            for fn in os.listdir(d):
                m=re.search(r'_(\d{4})\.dat$',fn)
                if m: ids.add(int(m.group(1)))
            break
    return sorted(ids)

ids = present_ids(pdf)
err2 = np.zeros_like(central_cross_section)
k = 0
while True:
    ip, im = 1+2*k, 2+2*k
    if ip in ids and im in ids:
        F2p,FLp,xF3p = extract_Fs(lhapdf.mkPDF(pdf,ip))
        F2m,FLm,xF3m = extract_Fs(lhapdf.mkPDF(pdf,im))
        sp = compute_cross_section(F2p, FLp, xF3p,
               RGE_df["x"].to_numpy(), RGE_df["y"].to_numpy(), RGE_df["Q2"].to_numpy(),
               alpha, True, wrt="dx")
        sm = compute_cross_section(F2m, FLm, xF3m,
               RGE_df["x"].to_numpy(), RGE_df["y"].to_numpy(), RGE_df["Q2"].to_numpy(),
               alpha, True, wrt="dx")
        err2 += ((sp - sm)/2.0)**2
        k += 1
    else:
        break

sigma_error68 = np.sqrt(err2) / 1.645   # nCTEQ15HIX: 90% → 68%


[14:25:26] INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=374892;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=827694;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

LHAPDF 6.5.5 loading /home/ryan/yadism/LHAPDF-6.5.5/install/share/LHAPDF/nCTEQ15HIX_FullNuc_2_1/nCTEQ15HIX_FullNuc_2_1_0001.dat
nCTEQ15HIX_FullNuc_2_1 PDF set, member #1, version 3; LHAPDF ID = 110051


           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=225148;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=855397;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=440589;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=62969;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

LHAPDF 6.5.5 loading /home/ryan/yadism/LHAPDF-6.5.5/install/share/LHAPDF/nCTEQ15HIX_FullNuc_2_1/nCTEQ15HIX_FullNuc_2_1_0002.dat
nCTEQ15HIX_FullNuc_2_1 PDF set, member #2, version 3; LHAPDF ID = 110052


...

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=349729;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=473099;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

LHAPDF 6.5.5 loading /home/ryan/yadism/LHAPDF-6.5.5/install/share/LHAPDF/nCTEQ15HIX_FullNuc_2_1/nCTEQ15HIX_FullNuc_2_1_0042.dat
nCTEQ15HIX_FullNuc_2_1 PDF set, member #42, version 3; LHAPDF ID = 110092


[14:27:09] INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=653;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=133916;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

           INFO     Strong Coupling: a_s(µ_R^2=8317.440000)^(nf=5)=0.009390=0.118000/(4π)          ]8;id=111125;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py\couplings.py]8;;\:]8;id=798192;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/couplings.py#464\464]8;;\

           INFO     Atlas [0.00e+00 - 2.28e+00 - 2.42e+01 - 2.98e+04 - inf], ref=8317.44 @ 5        ]8;id=987190;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py\matchings.py]8;;\:]8;id=529921;file:///home/ryan/yadism/yadism_0.13.6_venv/lib/python3.10/site-packages/eko/matchings.py#72\72]8;;\

#### Save predictions to a CSV file with the data

In [8]:
RGE_df["sigma_yadism_pb_per_GeV2"]     = central_cross_section
RGE_df["sigma_yadism_pdf_err68"]       = sigma_error68

# Save merged table (optional)
RGE_df.to_csv(f"{RGE_file_name}_withyadism.csv", index=False)
RGE_df.head()

,x,Q2,absolute_cross_sections,absolute_cross_sections_errors,y,sigma_yadism_pb_per_GeV2,sigma_yadism_pdf_err68
0,0.072,1.027366,101.316757,19.147067,0.721138,454167.020414,0.0
1,0.072,1.083596,193.680132,24.597401,0.760607,411860.348799,0.0
2,0.072,1.142903,355.690680,30.388706,0.802237,372667.954513,0.0
3,0.072,1.205456,507.723360,32.047214,0.846145,336574.763949,0.0
4,0.072,1.271433,118.273001,12.828508,0.892456,303549.283038,0.0
